# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torchsummary import summary

In [11]:
#Attempt at a dynamic CNN model creater class, probably need to add maxpool and relu for final edition
class Dynamic_CNN(nn.Module):
    def __init__(self, in_dim, out_dim, layers):
        super(Dynamic_CNN, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.layers = layers
        
        self.layers = nn.ModuleList()
        in_chan = in_dim

        for out_chan, kernel_size in layers:
            conv_layer = nn.Sequential(
                            nn.Conv2d(in_chan, out_chan, kernel_size),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=kernel_size))
            self.layers.append(conv_layer)
            in_chan = out_chan
        
        self.fc = nn.Linear(in_chan, out_dim)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = nn.functional.relu(x)
        
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [31]:
in_dim = 20
out_dim = 10
hidden_layers = [(100, 3), (100, 3)]

testmodel = Dynamic_CNN(in_dim, out_dim, hidden_layers)
summary(testmodel,(in_dim,18,18))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 100, 16, 16]          18,100
              ReLU-2          [-1, 100, 16, 16]               0
         MaxPool2d-3            [-1, 100, 5, 5]               0
            Conv2d-4            [-1, 100, 3, 3]          90,100
              ReLU-5            [-1, 100, 3, 3]               0
         MaxPool2d-6            [-1, 100, 1, 1]               0
            Linear-7                   [-1, 10]           1,010
Total params: 109,210
Trainable params: 109,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.42
Params size (MB): 0.42
Estimated Total Size (MB): 0.87
----------------------------------------------------------------
